In [2]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3



# SQLite to Store Financial Analysis on the local drive


In [ ]:
<!-- PELICAN_BEGIN_SUMMARY -->

After merging and reshaping the datasets, we can use SQLite to safely storing, organizing and manipulating data in smaller environments.
SQLite is a flexible database can do real work in real business environments. 
This database tool can be useful for accountants to fo financial analysis.


<!-- PELICAN_END_SUMMARY -->

from book "Using SQLite by Jay A. Kreibich"

what is SQLite: In the simplest terms, SQLite is a public-domain software package that provides a
relational database management system, or RDBMS. Relational database systems are
used to store user-defined records in large tables. In addition to data storage and management, a database engine can process complex query commands that combine data
from multiple tables to generate reports and data summaries. Other popular RDBMS
products include Oracle Database, IBM’s DB2, and Microsoft’s SQL Server on the
commercial side, with MySQL and PostgreSQL being popular open source products.
The “Lite” in SQLite does not refer to its capabilities. Rather, SQLite is lightweight
when it comes to setup complexity, administrative overhead, and resource usage

## What is DB Brower for SQLite
DB Browser for SQLite is a high quality, visual, open source tool to create, design, and edit database files compatible with SQLite.

It is for users and developers wanting to create databases, search, and edit data. It uses a familiar spreadsheet-like interface,
and you don't need to learn complicated SQL commands.

This program is not a visual shell for the sqlite command line tool.



## Why use SQLite to read Excel File?
- Sort data on several fields
- Combining data from several sheets 
- Install Scientific Python to convert Excel file in SQLite database in just 8 lines of code
- Use free GUI tools to run SQL queries on database
- Each sheet of Excel file is converted in a table
- Sheet name can use underscore, but no whitespace for legal SQLite table name
- The first row in each sheet should have legal column names. Here is conversion code:



In [3]:
#Name of Excel xlsx file. SQLite database will have the same name and extension .db
filename="data/Premium_Loss" 
con=sqlite3.connect(filename+".db")
wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
#     wb[sheet].to_sql(sheet,con, index=False)
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')
con.commit()
con.close()

C:\apps\python36\python-3.6.3.amd64\lib\site-packages\pandas\core\generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [5]:
## Install DB Brower for SQLite

## View Excel File as Table in SQLite

<img src="files/image.png">

## Export SQLite Database To a CSV File


SyntaxError: invalid syntax (<ipython-input-5-f51ee6f7bdc3>, line 5)